In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import sqlite3
import pandas as pd

# File paths
file_2019 = '/kaggle/input/trip-data/Divvy_Trips_2019_Q1/Divvy_Trips_2019_Q1.csv'
file_2020 = '/kaggle/input/trip-data/Divvy_Trips_2020_Q1 (1)/Divvy_Trips_2020_Q1.csv'

# Load CSVs into pandas
data_2019 = pd.read_csv(file_2019)
data_2020 = pd.read_csv(file_2020)

# Connect to SQLite database
conn = sqlite3.connect('bike_share.db')

# Import data into SQLite tables
data_2019.to_sql('trips_2019', conn, if_exists='replace', index=False)
data_2020.to_sql('trips_2020', conn, if_exists='replace', index=False)

print("Data imported into SQLite database.")


Solution_case_study1 Cyclitic Bike-share Case study1
 AHMADI OTHMAN 2024-12-21
 Hi everyone, I’ve been working on the Google Data Analytics Professional Certificate through Coursera. This capstone project is the final project in my Google Data Analytics Professional Certificate course.
Business Task Cyclistic aims to increase annual memberships by understanding differences in bike usage between casual riders and members. Insights from this analysis will inform marketing strategies to convert casual riders into annual members. In this case study, we tasked to analyze a public dataset for a fictional company provided by the course. I used Python programming language and Kaggle motebook for data analysis and visulisation
Data Sources:
o	Cyclistic bike-share trip data for 2019-2020 ( CSV files). 
o	 Data included fields such as trip start/end times, ride length, and rider type       (casual vs. member).
o	 The following data analysis steps will be followed: Ask, Prepare, Process, Analyze, Share, Act.
1.	Ask
 The questions that need to be answered are:
•	How do annual members and casual riders use Cyclistic bikes differently? Why would casual riders buy Cyclistic annual memberships?
•	 How can Cyclistic use digital media to influence casual riders to become members? Analysis for the three guiding questions:
•	How do annual members and casual riders use Cyclistic bikes differently? To analyze this, we can focus on the usertype, tripduration, and start_time/end_time columns:
2.	Cleaning data 
Data cleaning included converting time columns to datetime formats, removing anomalies in ride lengths, and ensuring consistent formatting across datasets.
3.	Analysis for the first guiding question: 

How do annual members and casual riders use Cyclistic bikes differently?
To analyze this, we focused on the following:
•	Trip Frequency: Group trips by user type to compare the number of trips.
•	Trip Duration: Analyze the trip duration to understand usage patterns.
•	Time of Use: Identify trends based on the hour of the day and day of the week.
________________________________________


In [ ]:
import sqlite3
import pandas as pd

# File paths
file_2019 = '/kaggle/input/trip-data/Divvy_Trips_2019_Q1/Divvy_Trips_2019_Q1.csv'
file_2020 = '/kaggle/input/trip-data/Divvy_Trips_2020_Q1 (1)/Divvy_Trips_2020_Q1.csv'

# Load CSVs into pandas
data_2019 = pd.read_csv(file_2019)
data_2020 = pd.read_csv(file_2020)

# Connect to SQLite database
conn = sqlite3.connect('bike_share.db')

# Import data into SQLite tables
data_2019.to_sql('trips_2019', conn, if_exists='replace', index=False)
data_2020.to_sql('trips_2020', conn, if_exists='replace', index=False)

print("Data imported into SQLite database.")

# Step 2: Check total number of rows
query = """
SELECT 
    'trips_2019' AS table_name, COUNT(*) AS total_rows 
FROM trips_2019
UNION ALL
SELECT 
    'trips_2020', COUNT(*) 
FROM trips_2020;
"""
print(pd.read_sql_query(query, conn))


In [ ]:
conn.execute("DROP TABLE IF EXISTS combined_trips;")
print(pd.read_sql_query("PRAGMA table_info(trips_2019);", conn))
print(pd.read_sql_query("PRAGMA table_info(trips_2020);", conn))


In [ ]:
import pandas as pd
import sqlite3
import numpy as np

# **Step 1: Load Data**
file_2019 = '/kaggle/input/trip-data/Divvy_Trips_2019_Q1/Divvy_Trips_2019_Q1.csv'
file_2020 = '/kaggle/input/trip-data/Divvy_Trips_2020_Q1 (1)/Divvy_Trips_2020_Q1.csv'

# Load CSVs into pandas
data_2019 = pd.read_csv(file_2019)
data_2020 = pd.read_csv(file_2020)

# Harmonize column names
data_2019.rename(columns={
    'start_time': 'started_at',
    'end_time': 'ended_at',
    'from_station_name': 'start_station_name',
    'to_station_name': 'end_station_name',
    'usertype': 'member_casual'
}, inplace=True)

# Ensure `member_casual` consistency
data_2019['member_casual'] = data_2019['member_casual'].replace({
    'Subscriber': 'member',
    'Customer': 'casual'
})

# Create ride_length and day_of_week for both datasets
for df in [data_2019, data_2020]:
    df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
    df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce')
    df['ride_length'] = (df['ended_at'] - df['started_at']).dt.total_seconds() / 60  # in minutes
    df['day_of_week'] = df['started_at'].dt.dayofweek + 1  # Monday=1, Sunday=7

# Filter out invalid ride lengths
data_2019 = data_2019[(data_2019['ride_length'] > 0) & (data_2019['ride_length'] < 1440)]
data_2020 = data_2020[(data_2020['ride_length'] > 0) & (data_2020['ride_length'] < 1440)]

# **Step 2: Store Data in SQLite**
conn = sqlite3.connect('bike_share.db')
data_2019.to_sql('trips_2019', conn, if_exists='replace', index=False)
data_2020.to_sql('trips_2020', conn, if_exists='replace', index=False)
query = "DROP TABLE IF EXISTS combined_trips;"
conn.execute(query)  # Drop the table if it exists

# Connect to SQLite database
conn = sqlite3.connect('bike_share.db')

# Add ride_length and day_of_week columns
for df in [data_2019, data_2020]:
    df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
    df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce')
    df['ride_length'] = (df['ended_at'] - df['started_at']).dt.total_seconds() / 60
    df['day_of_week'] = df['started_at'].dt.dayofweek + 1  # Monday=1, Sunday=7

# Save to SQLite
data_2019.to_sql('trips_2019', conn, if_exists='replace', index=False)
data_2020.to_sql('trips_2020', conn, if_exists='replace', index=False)

# Drop the existing table if it exists
conn.execute("DROP TABLE IF EXISTS combined_trips;")

# Combine tables with aligned columns
query = """
CREATE TABLE combined_trips AS
SELECT 
    '2019' AS year,
    trip_id,
    started_at,
    ended_at,
    bikeid,
    tripduration,
    from_station_id,
    start_station_name,
    to_station_id,
    end_station_name,
    member_casual,
    gender,
    birthyear,
    ride_length,
    day_of_week,
    NULL AS rideable_type,
    NULL AS start_lat,
    NULL AS start_lng,
    NULL AS end_lat,
    NULL AS end_lng
FROM trips_2019
UNION ALL
SELECT 
    '2020' AS year,
    ride_id AS trip_id,
    started_at,
    ended_at,
    NULL AS bikeid,
    NULL AS tripduration,
    start_station_id AS from_station_id,
    start_station_name,
    end_station_id AS to_station_id,
    end_station_name,
    member_casual,
    NULL AS gender,
    NULL AS birthyear,
    ride_length,
    day_of_week,
    rideable_type,
    start_lat,
    start_lng,
    end_lat,
    end_lng
FROM trips_2020;
"""
conn.execute(query)
print("Tables combined into 'combined_trips'.")



In [ ]:
import pandas as pd
import sqlite3
import numpy as np


# Summary statistics for ride_length
query = """
SELECT 
    member_casual,
    AVG(ride_length) AS avg_length,
    MAX(ride_length) AS max_length,
    COUNT(*) AS total_rides
FROM combined_trips
GROUP BY member_casual;
"""
print(pd.read_sql_query(query, conn))

# **Step 4: Create Pivot Tables**
# Average ride length for members and casual riders
pivot_avg_length = pd.read_sql_query("""
SELECT 
    member_casual,
    AVG(ride_length) AS avg_length
FROM combined_trips
GROUP BY member_casual;
""", conn)

# Average ride length by day_of_week and user type
pivot_avg_length_by_day = pd.read_sql_query("""
SELECT 
    day_of_week,
    member_casual,
    AVG(ride_length) AS avg_length
FROM combined_trips
GROUP BY day_of_week, member_casual;
""", conn)

# Number of rides by day_of_week
pivot_count_rides_by_day = pd.read_sql_query("""
SELECT 
    day_of_week,
    member_casual,
    COUNT(*) AS total_rides
FROM combined_trips
GROUP BY day_of_week, member_casual;
""", conn)

# **Step 5: Export Pivot Tables**
pivot_avg_length.to_csv('/kaggle/working/pivot_avg_length.csv', index=False)
pivot_avg_length_by_day.to_csv('/kaggle/working/pivot_avg_length_by_day.csv', index=False)
pivot_count_rides_by_day.to_csv('/kaggle/working/pivot_count_rides_by_day.csv', index=False)

# **Step 6: Visualizations**
import matplotlib.pyplot as plt
import seaborn as sns

# Visualization 1: Average ride length by user type
plt.figure(figsize=(8, 6))
sns.barplot(data=pivot_avg_length, x='member_casual', y='avg_length', palette='Set2')
plt.title('Average Ride Length by User Type')
plt.xlabel('User Type')
plt.ylabel('Average Ride Length (Minutes)')
plt.grid(axis='y')
plt.savefig('/kaggle/working/avg_ride_length_by_user.png')
plt.show()

# Visualization 2: Average ride length by day_of_week
pivot_avg_length_by_day = pivot_avg_length_by_day.pivot(index='member_casual', columns='day_of_week', values='avg_length')
pivot_avg_length_by_day.plot(kind='bar', figsize=(10, 6), colormap='Set1')
plt.title('Average Ride Length by Day of the Week and User Type')
plt.xlabel('User Type')
plt.ylabel('Average Ride Length (Minutes)')
plt.legend(title='Day of Week', loc='upper left')
plt.grid(axis='y')
plt.savefig('/kaggle/working/avg_ride_length_by_day.png')
plt.show()

# Visualization 3: Number of rides by day_of_week
pivot_count_rides_by_day = pivot_count_rides_by_day.pivot(index='member_casual', columns='day_of_week', values='total_rides')
pivot_count_rides_by_day.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='Accent')
plt.title('Number of Rides by Day of the Week and User Type')
plt.xlabel('User Type')
plt.ylabel('Number of Rides')
plt.legend(title='Day of Week', loc='upper left')
plt.grid(axis='y')
plt.savefig('/kaggle/working/ride_counts_by_day.png')
plt.show()


Visualizations and Key Findings
1. Number of Rides by Day of the Week
Interpretation: Members show consistent activity across weekdays, while casual riders dominate on weekends, reflecting leisure usage.
2. Average Ride Duration by Day
Interpretation: Casual riders have longer rides, especially on weekends, whereas members maintain consistent ride durations.
3. Ride Duration Distribution
Interpretation: Boxplots show casual riders have higher variability and longer outliers in ride durations compared to members.


This bar chart visualizes the average ride length for different user types (casual and member) across each day of the week. Here's the analysis and some recommendations based on the data:
________________________________________
Analysis
1.	Casual Riders:
o	Have longer ride lengths consistently across the week, with peaks on weekends (Day 6 and Day 7).
o	This trend suggests that casual riders use the service more for leisure and recreational activities during their free time.
2.	Members:
o	Exhibit steady ride lengths throughout the week, with only slight variations.
o	This indicates members predominantly use the service for consistent, routine purposes like commuting, regardless of the day.
3.	Day-Specific Trends:
o	The ride lengths for casual users increase significantly on weekends, while members’ ride lengths show little change.
o	Casual users' highest ride lengths appear on Day 6 and Day 7 (Saturday and Sunday).
________________________________________
Recommendations
1.	Service Optimization:
o	Allocate more bikes and staff at high-demand stations on weekends to accommodate the surge in casual riders.
o	For members, maintain steady availability during weekdays, especially around commuting hours.
2.	Marketing and Engagement:
o	For casual users: Introduce weekend-specific offers, such as group discounts or extended ride time without extra charges.
o	For members: Incentivize weekend usage by offering small rewards or loyalty points for weekend rides.


RECOMMANDATIONS
1.	Targeted Promotions Based on Trip Duration: Cyclistic could create digital campaigns targeting casual riders who make frequent short trips, emphasizing that an annual membership would provide them with better value and convenience (particularly if they take multiple trips in a given month).
2.	Incentivizing Frequent Riders: For casual riders who tend to use bikes consistently over a short period (e.g., several trips in a week or month), digital media could highlight how switching to a membership would eliminate the hassle of paying for individual rides and allow them to enjoy more rides at no extra cost. 
3.	Promote annual memberships to occasional users by highlighting the benefits for recreational (weekend) activities. Target members with commuting offers. Integrate these analytics into specific marketing campaigns through digital channels. 
4.	Push Notifications/Email Campaigns: Casual riders who have frequent trips but not necessarily the longest durations might be enticed by a special offer that shows how much they would save with a membership over time. This can be done through push notifications or emails, with a clear breakdown of savings based on their tripduration patterns.
